In [91]:
from pyscf import gto, scf, ao2mo, fci
from GeneralisedWick import *
from scipy.optimize import fsolve

In [92]:
def commutator(operator1, operator2):
    return operator1 * operator2 + (-1) * operator2 * operator1

In [93]:
def BCHSimilarityTransform(H, T, order):
    result = H
    for k in range(order):
        nestedCommutator = H
        for i in range(k + 1):
            nestedCommutator = commutator(nestedCommutator, T)
        result += (1 / factorial(k + 1)) * nestedCommutator
    return result

In [94]:
def projectionManifold(excitationLevel):
    return Tensor('\Phi', ['h'] * excitationLevel, ['p'] * excitationLevel)

In [95]:
fDiagrams = [Tensor('f', ['p'], ['p']), Tensor('f', ['p'], ['h']), Tensor('f', ['h'], ['p']), Tensor('f', ['h'], ['h'])]
vDiagrams = [Tensor('v', ['p', 'p'], ['p', 'p']), Tensor('v', ['h', 'h'], ['h', 'h']), Tensor('v', ['h', 'p'], ['p', 'h']), Tensor('v', ['p', 'h'], ['p', 'p']), Tensor('v', ['h', 'h'], ['h', 'p']), Tensor('v', ['p', 'p'], ['p', 'h']), Tensor('v', ['h', 'p'], ['h', 'h']), Tensor('v', ['p', 'p'], ['h', 'h']), Tensor('v', ['h', 'h'], ['p', 'p'])]
t1 = Tensor('t1', ['p'], ['h'])
t2 = Tensor('t2', ['p', 'p'], ['h', 'h'])

In [96]:
projectionSingles = projectionManifold(1)
projectionDoubles = projectionManifold(2)

In [97]:
NormalOrderedFock = sum(fDiagrams)
NormalOrdered2BodyH = (1./4.) * vDiagrams[0] + (1./4.) * vDiagrams[1] + vDiagrams[2] + (1./2.) * vDiagrams[3] + (1./2.) * vDiagrams[4] + (1./2.) * vDiagrams[5] + (1./2.) * vDiagrams[6] + (1./4.) * vDiagrams[7] + (1./4.) * vDiagrams[8]
NormalOrderedHamiltonian = NormalOrderedFock + NormalOrdered2BodyH

In [98]:
#CCDtransformH = sum([BCHSimilarityTransform(d, (1./4.) * t2, 4) for d in fDiagrams] + [BCHSimilarityTransform((1./2.) * d, (1./4.) * t2, 4) for d in vDiagrams])
CCDtransformH = BCHSimilarityTransform(NormalOrderedHamiltonian, (1./4.) * t2, 4)

In [99]:
projectedCCDtransformH = projectionDoubles * CCDtransformH

In [10]:
prod = vDiagrams[8] * t2
print(prod)

1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}


In [11]:
copyProd = copy(prod)

In [12]:
prod.vertexList


In [13]:
copyProd.vertexList

In [15]:
print(prod)
print(evaluateWick(prod, spinFree=False))

1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{2}p_{3}}^{h_{2}h_{3}}
1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{1}}
 + -1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{1}h_{0}}
 + 1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{1}h_{0}}


In [16]:
print(evaluateWick((vDiagrams[8] * t2), spinFree=False))

1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{1}}
 + -1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{1}h_{0}}
 + 1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{1}h_{0}}


In [100]:
CCDEnergyEquation = evaluateWick(CCDtransformH, spinFree=False)

In [101]:
print(CCDEnergyEquation)

0.0625 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -0.0625 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{1}}
 + -0.0625 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{1}h_{0}}
 + 0.0625 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{1}h_{0}}


In [102]:
CCDAmplitudeEquation = evaluateWick(projectedCCDtransformH, spinFree=False)

In [103]:
print(CCDAmplitudeEquation)

0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{1}p_{0}}^{h_{0}h_{1}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{0}p_{1}}^{h_{1}h_{0}}
 + 0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{0}h_{1}}
 + 0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{0}h_{1}}
 + 0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{0}h_{1}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{0}h_{1}}
 + 0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{1}h_{0}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{1}h_{0}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{1}h_{0}}
 + 0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{1}h_{0}}
 + 0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{

In [21]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.UHF(mol)
mf.kernel()

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff.reshape((2,-1)), compact=False)

cisolver = fci.FCI(mol, mf.mo_coeff)

Norbs = mol.nao * 2
Nocc = sum(mf.nelec)
vacuum = [1 for i in range(Nocc)] + [0 for i in range (Norbs - Nocc)]

h2 = eri.reshape((Norbs, Norbs, Norbs, Norbs))

In [22]:
mf.mo_coeff

array([[[ 0.56092571, -1.10314488],
        [ 0.56092571,  1.10314488]],

       [[ 0.56092571, -1.10314488],
        [ 0.56092571,  1.10314488]]])

In [23]:
mf.get_hcore()

array([[-1.05506426, -0.83576253],
       [-0.83576253, -1.05506426]])

In [87]:
tuple([2 * dim for dim in mf.get_hcore().shape])

(4, 4)

In [104]:
def get_h1_spinorb(h1_orb):
    h1_spinorb = np.zeros(tuple([2 * dim for dim in h1_orb.shape]))
    for i in range(h1_orb.shape[0]):
        for j in range(h1_orb.shape[1]):
            h1_spinorb[2 * i, 2 * j] = h1_orb[i,j]
            h1_spinorb[2 * i + 1, 2 * j + 1] = h1_orb[i,j]
    return h1_spinorb

In [105]:
def get_h2_spinorb(h2_orb):
    h2_spinorb = np.zeros(tuple([2 * dim for dim in h2_orb.shape]))
    for i in range(h2_orb.shape[0]):
        for j in range(h2_orb.shape[1]):
            for k in range(h2_orb.shape[2]):
                for l in range(h2_orb.shape[3]):
                    h2_spinorb[2 * i, 2 * j, 2 * k, 2* l] = h2_orb[i,j,k,l]
                    h2_spinorb[2 * i + 1, 2 * j, 2 * k + 1, 2 * l] = h2_orb[i,j,k,l]
                    h2_spinorb[2 * i, 2 * j + 1, 2 * k, 2 * l + 1] = h2_orb[i,j,k,l]
                    h2_spinorb[2 * i + 1, 2 * j + 1, 2 * k + 1, 2 * l + 1] = h2_orb[i,j,k,l]
    return h2_spinorb

In [106]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

h1_orb = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff, compact=False)

cisolver = fci.FCI(mol, mf.mo_coeff)

Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1] * Nocc + [0] * (Norbs - Nocc)

h2_orb = eri.reshape((Norbs, Norbs, Norbs, Norbs), order='C')

h1 = get_h1_spinorb(h1_orb)
h2 = get_h2_spinorb(h2_orb)

vacuum = [1] * (2 * Nocc) + [0] * (2 * (Norbs - Nocc))

In [107]:
h1

array([[-1.18985062e+00,  0.00000000e+00,  2.60021255e-17,
         0.00000000e+00],
       [ 0.00000000e+00, -1.18985062e+00,  0.00000000e+00,
         2.60021255e-17],
       [-4.71423569e-17,  0.00000000e+00, -5.33749102e-01,
         0.00000000e+00],
       [ 0.00000000e+00, -4.71423569e-17,  0.00000000e+00,
        -5.33749102e-01]])

In [108]:
mf.mo_coeff.T.reshape(4,2).T

ValueError: cannot reshape array of size 4 into shape (4,2)

In [25]:
h1.reshape((4,4))

array([[-1.18985062e+00, -2.22044605e-16, -1.18985062e+00,
        -2.22044605e-16],
       [-1.18985062e+00, -2.22044605e-16, -1.18985062e+00,
        -2.22044605e-16],
       [ 2.34002041e+00,  4.44089210e-16,  2.34002041e+00,
         4.44089210e-16],
       [-2.34002041e+00,  4.44089210e-16, -2.34002041e+00,
         4.44089210e-16]])

In [28]:
eri.shape

(16, 16)

In [ ]:
h2

array([[[[ 0.65400951, -1.28620819,  0.65400951,  1.28620819],
         [-1.28620819,  2.52952208, -1.28620819, -2.52952208],
         [ 0.65400951, -1.28620819,  0.65400951,  1.28620819],
         [ 1.28620819, -2.52952208,  1.28620819,  2.52952208]],

        [[-1.28620819,  2.52952208, -1.28620819, -2.52952208],
         [ 2.52952208, -4.97468607,  2.52952208,  4.97468607],
         [-1.28620819,  2.52952208, -1.28620819, -2.52952208],
         [-2.52952208,  4.97468607, -2.52952208, -4.97468607]],

        [[ 0.65400951, -1.28620819,  0.65400951,  1.28620819],
         [-1.28620819,  2.52952208, -1.28620819, -2.52952208],
         [ 0.65400951, -1.28620819,  0.65400951,  1.28620819],
         [ 1.28620819, -2.52952208,  1.28620819,  2.52952208]],

        [[ 1.28620819, -2.52952208,  1.28620819,  2.52952208],
         [-2.52952208,  4.97468607, -2.52952208, -4.97468607],
         [ 1.28620819, -2.52952208,  1.28620819,  2.52952208],
         [ 2.52952208, -4.97468607,  2.52952208, 

In [ ]:
h1

array([[[[-1.18985062e+00, -2.22044605e-16],
         [-1.18985062e+00, -2.22044605e-16]],

        [[-1.18985062e+00, -2.22044605e-16],
         [-1.18985062e+00, -2.22044605e-16]]],


       [[[ 2.34002041e+00,  4.44089210e-16],
         [ 2.34002041e+00,  4.44089210e-16]],

        [[-2.34002041e+00,  4.44089210e-16],
         [-2.34002041e+00,  4.44089210e-16]]]])

In [ ]:
mf.mo_coeff.T.reshape(2, -1)

array([[ 0.56092571,  0.56092571],
       [-1.10314488,  1.10314488]])

In [ ]:
mf.mo_coeff

array([[ 0.56092571, -1.10314488],
       [ 0.56092571,  1.10314488]])

In [ ]:
eri

array([[6.54009511e-01, 1.11022302e-16, 8.32667268e-17, 6.45249427e-01],
       [1.17961196e-16, 1.87521981e-01, 1.87521981e-01, 1.11022302e-16],
       [1.04083409e-16, 1.87521981e-01, 1.87521981e-01, 5.55111512e-17],
       [6.45249427e-01, 2.22044605e-16, 1.38777878e-16, 6.78136184e-01]])

In [ ]:
h2 = eri.reshape((Norbs, Norbs, Norbs, Norbs))

In [110]:
h2

array([[[[6.54009511e-01, 0.00000000e+00, 1.11022302e-16,
          0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00],
         [8.32667268e-17, 0.00000000e+00, 6.45249427e-01,
          0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00]],

        [[0.00000000e+00, 6.54009511e-01, 0.00000000e+00,
          1.11022302e-16],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00],
         [0.00000000e+00, 8.32667268e-17, 0.00000000e+00,
          6.45249427e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00]],

        [[1.17961196e-16, 0.00000000e+00, 1.87521981e-01,
          0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00],
         [1.87521981e-01, 0.00000000e+00, 1.11022302e-16,
          0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
   

In [ ]:
eri[1,1]

0.18752198090270877

In [ ]:
h2[0,1,0,1]

0.18752198090270877

In [50]:
h1 = h1.reshape((4,4))

In [119]:
fDiagrams[0].getShape(vacuum)
fDiagrams[0].array = h1[Nocc:, Nocc:]

fDiagrams[1].getShape(vacuum)
fDiagrams[1].array = h1[Nocc:, :Nocc]

fDiagrams[2].getShape(vacuum)
fDiagrams[2].array = h1[:Nocc, Nocc:]

fDiagrams[3].getShape(vacuum)
fDiagrams[3].array = h1[:Nocc, :Nocc]

vDiagrams[0].getShape(vacuum)
vDiagrams[0].array = h2[Nocc:, Nocc:, Nocc:, Nocc:]

vDiagrams[1].getShape(vacuum)
vDiagrams[1].array = h2[:Nocc, :Nocc, :Nocc, :Nocc]

vDiagrams[2].getShape(vacuum)
vDiagrams[2].array = h2[:Nocc, Nocc:, Nocc:, :Nocc]

vDiagrams[3].getShape(vacuum)
vDiagrams[3].array = h2[Nocc:, :Nocc, Nocc:, Nocc:]

vDiagrams[4].getShape(vacuum)
vDiagrams[4].array = h2[:Nocc, :Nocc, :Nocc, Nocc:]

vDiagrams[5].getShape(vacuum)
vDiagrams[5].array = h2[Nocc:, Nocc:, Nocc:, :Nocc]

vDiagrams[6].getShape(vacuum)
vDiagrams[6].array = h2[:Nocc, Nocc:, :Nocc, :Nocc]

vDiagrams[7].getShape(vacuum)
vDiagrams[7].array = h2[Nocc:, Nocc:, :Nocc, :Nocc]

vDiagrams[8].getShape(vacuum)
vDiagrams[8].array = h2[:Nocc, :Nocc, Nocc:, Nocc:]

t1.getShape(vacuum)

t2.getShape(vacuum)

In [128]:
t2.getShape(vacuum)

In [59]:
print(CCDEnergyEquation)

0.125 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -0.125 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{1}}
 + -0.125 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{1}h_{0}}
 + 0.125 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{1}h_{0}}


In [120]:
CCDElectronicEnergy = contractTensorSum(CCDEnergyEquation)
print(CCDElectronicEnergy)
print(mf.energy_tot() + CCDElectronicEnergy)

0.0
-1.1026388111917238


In [33]:
print(CCDAmplitudeEquation)

-0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{0}h_{1}}
 + 0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{0}h_{1}}
 + 0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{0}h_{1}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{0}h_{1}}
 + 0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{1}h_{0}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{1}h_{0}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{1}h_{0}}
 + 0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{1}h_{0}}
 + 0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{1}h_{2}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{1}h_{2}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{2}h_{1}}
 + 0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{

In [121]:
len(CCDAmplitudeEquation.summandList)

660

In [122]:
testCCDAmplitudeContractions = copy(CCDAmplitudeEquation)

In [123]:
for summand in testCCDAmplitudeContractions.summandList:
    summand.vertexList.pop(0)
    summand.tensorList.pop(0)

In [124]:
print(testCCDAmplitudeContractions)

0.25 * v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -0.25 * v_{p_{1}p_{0}}^{h_{0}h_{1}}
 + -0.25 * v_{p_{0}p_{1}}^{h_{1}h_{0}}
 + 0.25 * v_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.25 * f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{0}h_{1}}
 + 0.25 * f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{0}h_{1}}
 + 0.25 * f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{0}h_{1}}
 + -0.25 * f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{0}h_{1}}
 + 0.25 * f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{1}h_{0}}
 + -0.25 * f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{1}h_{0}}
 + -0.25 * f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{1}h_{0}}
 + 0.25 * f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{1}h_{0}}
 + 0.25 * f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{1}h_{2}}
 + -0.25 * f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{1}h_{2}}
 + -0.25 * f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{2}h_{1}}
 + 0.25 * f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{2}h_{1}}
 + -0.25 * f_{h_{2}}^{h_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{2}}
 + 0.25 * f_{h_{2}}^{h_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{2}}
 + 0.25 * f_{h_{2}}^{h_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{0}}
 +

In [38]:
testCCDAmplitudeContractions.summandList[0].tensorList

In [146]:
print(t2.array)

[[[[1. 0.]
   [0. 0.]]

  [[0. 1.]
   [0. 0.]]]


 [[[0. 0.]
   [1. 0.]]

  [[0. 0.]
   [0. 1.]]]]


In [139]:
t2signature = np.zeros(t2.array.shape)

In [190]:
print(t2signature)

[[[[1. 0.]
   [0. 0.]]

  [[0. 1.]
   [0. 0.]]]


 [[[0. 0.]
   [1. 0.]]

  [[0. 0.]
   [0. 1.]]]]


In [189]:
for i in range(2):
    for j in range(2):
        for k in range(2):
            for l in range(2):
                t2signature[i,j,k,l] = (i == k) and (j == l)

In [191]:
t2.array = copy(t2signature)

In [192]:
t2.array *= -0.06883332874447325 * 2

In [72]:
t2.array = np.ones_like(t2.array)

In [62]:
t2.array = -0.06883332874447325 * np.zeros_like(t2.array)

In [56]:
fDiagrams[0].array.shape

(2, 2)

In [194]:
CCDResidual = contractTensorSum(testCCDAmplitudeContractions)
print(CCDResidual)

[[[[ 5.42101086e-20  0.00000000e+00]
   [ 0.00000000e+00  0.00000000e+00]]

  [[ 0.00000000e+00  3.02436341e-01]
   [-3.02436341e-01  0.00000000e+00]]]


 [[[ 0.00000000e+00 -3.02436341e-01]
   [ 3.02436341e-01  0.00000000e+00]]

  [[ 0.00000000e+00  0.00000000e+00]
   [ 0.00000000e+00  5.42101086e-20]]]]


In [57]:
CCDResidual = sum([getContractedArray(term)[0] for term in CCDAmplitudeEquation.summandList])

In [193]:
contractTensorSum(CCDEnergyEquation)

-0.022207332951274018

In [184]:
def testCCDamplitude(t):
    t2.array = t2signature * t
    print(t2.array)
    CCDResidual = contractTensorSum(testCCDAmplitudeContractions)
    print(CCDResidual)
    return CCDResidual.flatten()

In [212]:
def testCCDamplitudes(tFlat):
    t2.array = np.array(tFlat).reshape(t2.array.shape)
#    print(t2.array)
    CCDResidual = contractTensorSum(testCCDAmplitudeContractions)
#    print(CCDResidual)
    return CCDResidual.flatten()

In [197]:
print(CCDResidual[0,1,0,1])

0.30243634143099224


In [213]:
testCCDamplitudes(np.ones(t2.array.shape))
amplitude = fsolve(testCCDamplitudes, np.ones(t2.array.shape))

In [216]:
t2.array

array([[[[ 1.38149814e+00,  5.91661232e+01],
         [-8.27170599e-01, -2.65580082e+00]],

        [[ 8.21915789e+00,  6.10002283e-01],
         [ 1.92619720e+00, -1.35567072e+01]]],


       [[[-2.79303892e+01,  1.93814618e+00],
         [ 5.96171535e-01, -3.54084854e+02]],

        [[ 2.35508808e+02,  3.12488812e+02],
         [ 8.82840158e+01, -9.39341119e+02]]]])

In [217]:
contractTensorSum(CCDEnergyEquation)

-0.21439779832272166

In [215]:
amplitude.reshape(t2.array.shape)

array([[[[ 1.38149814e+00,  5.91661262e+01],
         [-8.27170582e-01, -2.65580086e+00]],

        [[ 8.21915828e+00,  6.10002293e-01],
         [ 1.92619720e+00, -1.35567077e+01]]],


       [[[-2.79303917e+01,  1.93814618e+00],
         [ 5.96171530e-01, -3.54084851e+02]],

        [[ 2.35508831e+02,  3.12488799e+02],
         [ 8.82840149e+01, -9.39341121e+02]]]])

In [171]:
amplitude = fsolve(testCCDamplitude, 0)

[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]]
[ 0.          0.          0.          0.          0.          0.32262471
 -0.32262471  0.          0.         -0.32262471  0.32262471  0.
  0.          0.          0.          0.        ]
[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]]
[ 0.          0.          0.          0.          0.          0.32262471
 -0.32262471  0.          0.         -0.32262471  0.32262471  0.
  0.          0.          0.          0.        ]
[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]]
[ 0.          0.          0.          0.          0.          0.32262471
 -0.32262471  0.          0.         -0.32262471  0.32262471  0.
  0.          0.          0.          0.        ]


ValueError: The array returned by a function changed size between calls

In [154]:
amplitude

array([0.])

In [155]:
t2.array

array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]])